In [24]:
import numpy as np
import dask
import dask.array as da
%load_ext line_profiler

In [2]:
da.transpose

<function numpy.core.fromnumeric.transpose>

In [3]:
def dcg(A, b, tol=1e-8, maxiter=500, x0=None, preconditioner=None, verbose=0, client=None):
    """ Conjugate gradient
        
        Parameters
        ----------
        A: array-like
        b: array-like
        tol: float
        
        Returns
        -------
        x: array-like
        iters: int
        resnorm: float
        
        Find x such that

            Ax = b
        
        for square, symmetric A.

        If a preconditioner M is provided, solve the left-preconditioned
        equivalent problem,

            M(Ax - b) = 0
    """
    print_iter = max(1, maxiter / 10**verbose)

    A, b, M = dask.persist(A, b, preconditioner)

    if x0 is None:
        r = 1 * b
        x = 0 * b
    else:
        r = 1 * b - A.dot(x0)
        x = x0

    Mr = r if M is None else M.dot(r)
    p = Mr
    resnrm2 = r.dot(Mr)

    x, r, p, resnrm2 = dask.persist(x, r, p, resnrm2)
    (resnrm2,) = dask.compute(resnrm2)
    if resnrm2**0.5 < tol:
        return x, 0, resnrm2**0.5

    for k in range(maxiter):
        ox, ores, op, oresnrm2 = x, r, p, resnrm2

        Ap = A.dot(p)
        alpha = resnrm2 / p.dot(Ap)
        x = ox + alpha * p
        r = ores - alpha * Ap
        Mr = r if M is None else M.dot(r)
        resnrm2 = r.dot(Mr)

        x, r, resnrm2 = dask.persist(x, r, resnrm2)
        (resnrm2,) = dask.compute(resnrm2)

        if resnrm2**0.5 < tol:
            break
        elif (k + 1) % print_iter == 0:
            print("ITER: {:5}\t||Ax -  b||_2: {}".format(k + 1, resnrm2**0.5))

        p = Mr + (resnrm2 / oresnrm2) * op
        x, r, resnrm2, p= dask.persist(x, r, resnrm2, p)

        (p,) = dask.persist(p)

    return x, k + 1, resnrm2**0.5


## Numpy array testing

In [4]:
m = 400
n = 300
mc = 100
nc = 100

In [5]:
A = np.random.random((m, n))
x = np.random.random(n)
b = np.random.random(n)
rho = 1.

In [6]:
Asymm_unregularized = A.T.dot(A)

In [7]:
xcg, iters, res = dcg(Asymm_unregularized, b, verbose=5)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITER:     1	||Ax -  b||_2: 5.954936059837789
ITER:     2	||Ax -  b||_2: 4.515756617141952
ITER:     3	||Ax -  b||_2: 3.8234304419539065
ITER:     4	||Ax -  b||_2: 3.1257236167859337
ITER:     5	||Ax -  b||_2: 2.826238571725316
ITER:     6	||Ax -  b||_2: 2.7010391627856403
ITER:     7	||Ax -  b||_2: 27.248706020690978
ITER:     8	||Ax -  b||_2: 2.4236458589838366
ITER:     9	||Ax -  b||_2: 2.109629200669731
ITER:    10	||Ax -  b||_2: 1.8368963564816423
ITER:    11	||Ax -  b||_2: 1.546173460213795
ITER:    12	||Ax -  b||_2: 1.2787313302121377
ITER:    13	||Ax -  b||_2: 14.25466796972313
ITER:    14	||Ax -  b||_2: 1.0842027975920305
ITER:    15	||Ax -  b||_2: 0.9374326412471657
ITER:    16	||Ax -  b||_2: 0.776259862517466
ITER:    17	||Ax -  b||_2: 0.6679691094262388
ITER:    18	||Ax -  b||_2: 0.5387969300666192
ITER:    19	||Ax -  b||_2: 5.186177207176721
ITER:    20	||Ax -  b||_2: 0.4421667024435945
ITER:    21	||Ax -  b||_2: 0.3773210137745667
ITER:    22	||Ax -  b||_2: 0.3051220328485

In [8]:
Asymm_weakreg = A.T.dot(A) + 0.1 * np.eye(n)

In [9]:
xcg, iters, res = dcg(Asymm_weakreg, b)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:  135
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.45759963413295e-10


In [10]:
Asymm = A.T.dot(A) + np.eye(n)

In [11]:
xcg, iters, res = dcg(Asymm, b, verbose=1)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITER:    50	||Ax -  b||_2: 0.0035497162964003496
ITERS:   96
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.335360348776144e-10


# Preconditioning

In [12]:
xcg, iters, res = dcg(Asymm, b, preconditioner=np.eye(n))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   96
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.335360348776144e-10


In [13]:
class JacobiPrecond(object):
    """ Build a Jacobi diagonal preconditioner P = inv(diag(A))
    """
    def __init__(self, A_symmetric):
        self.d = np.reciprocal(np.diag(A_symmetric))
    def dot(self, v):
        return self.d * v

In [14]:
xcg, iters, res = dcg(Asymm, b, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   86
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.192065024253674e-10


## Warm start
Let $x_{cg}$ be the solution to $Ax = b$, then set $b_{new} = b + b'$ for some (small) perturbation $b'$, and re-solve the system with and without providing $x_{cg}$ as an initial guess. 

We also compare the effects of warm starting with those of using a preconditioner.

In [15]:
pct_perturb = 5.
perturb = np.random.random(n) - 0.5
perturb = (perturb / np.linalg.norm(perturb)) * (pct_perturb / 100.) * np.linalg.norm(b) 
b_new = b + perturb

In [16]:
xwarm, iters, res = dcg(Asymm, b_new, x0=xcg, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   74
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.647682906277335e-10


In [17]:
xcold, iters, res = dcg(Asymm, b_new, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   86
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.958555186430752e-10


In [18]:
xwarm_noprecon, iters, res = dcg(Asymm, b_new, x0=xcg)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   82
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.74764364839337e-10


In [19]:
xcold_noprecon, iters, res = dcg(Asymm, b_new)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   96
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.766699953364872e-10


|Conditions                  | Iterations |
|:---------------------------|------------|
|Cold start                  | 93         |
|Cold start + preconditioner | 84         |
|Warm start                  | 82         |
|Warm start + preconditioner | 72         |

## Dask vs. numpy arrays

In [20]:
c = n/2

In [21]:
Ad = da.from_array(Asymm, chunks=c)
bd = da.from_array(b, chunks=c)

In [25]:
%lprun -f dcg dcg(Ad, b)

In [26]:
%lprun -f dcg dcg(Asymm, b)

For $A\in R^{300\times 300}$, the CG runtime is around 6ms using ``numpy`` arrays. When we switch to ``dask`` arrays, the results (solution $x$, iteration count $k$, and residual $\|Ax-b\|_2$) are all the same, but the call now takes around 2s.